In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
import pickle
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [2]:
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)

## Dataset loading

In [3]:
base_dir = '/home/nur/Documents/Selise/dataset/dataset/'

In [4]:
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [5]:
print(train_dir)
print(test_dir)

/home/nur/Documents/Selise/dataset/dataset/train
/home/nur/Documents/Selise/dataset/dataset/test


## Data Augmentation and Generators

In [6]:
train_datagen = ImageDataGenerator(
    rescale = 1./255., 
    rotation_range = 40, 
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.2, 
    zoom_range = 0.2, 
    horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    batch_size = BATCH_SIZE, 
    class_mode = 'categorical', 
    target_size = TARGET_SIZE,
    subset = 'training')

Found 4800 images belonging to 4 classes.


In [8]:
validation_generator = test_datagen.flow_from_directory(
    test_dir,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    target_size = TARGET_SIZE)

Found 1600 images belonging to 4 classes.


In [9]:
# val_generator = test_datagen.flow_from_directory(
#     test_dir, 
#     batch_size = BATCH_SIZE, 
#     class_mode = 'categorical', 
#     target_size = TARGET_SIZE,
#     subset = 'validation')

## VGG

In [10]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint

In [11]:
base_model = VGG19(input_shape = (224, 224, 3),
                   include_top = False,
                   weights = 'imagenet')

80142336/80134624 [==============================] - 601s 8us/step


In [12]:
for layer in base_model.layers:
    layer.trainable = False

In [13]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(4, activation='softmax')(x)


model = tf.keras.models.Model(base_model.input, x)

In [14]:
model.compile(optimizer = tf.keras.optimizers.SGD(lr=0.0001), loss = 'categorical_crossentropy',metrics = ['acc'])

In [15]:
filepath="weights-improvement-{epoch:02d}-{acc:.2f}-{val_acc:.2f}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [17]:
vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 50, epochs = 10, callbacks=callbacks_list, verbose=1)

Epoch 1/10
50/50 [==============================] - ETA: 0s - loss: 1.6115 - acc: 0.2606 
Epoch 00001: val_loss improved from inf to 1.33707, saving model to weights-improvement-01-0.26-0.34-1.34.hdf5
50/50 [==============================] - 1596s 32s/step - loss: 1.6115 - acc: 0.2606 - val_loss: 1.3371 - val_acc: 0.3375
Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 1.4764 - acc: 0.2969 
Epoch 00002: val_loss improved from 1.33707 to 1.26985, saving model to weights-improvement-02-0.30-0.44-1.27.hdf5
50/50 [==============================] - 1566s 31s/step - loss: 1.4764 - acc: 0.2969 - val_loss: 1.2698 - val_acc: 0.4381
Epoch 3/10
50/50 [==============================] - ETA: 0s - loss: 1.4241 - acc: 0.3394 
Epoch 00003: val_loss improved from 1.26985 to 1.21759, saving model to weights-improvement-03-0.34-0.49-1.22.hdf5
50/50 [==============================] - 1560s 31s/step - loss: 1.4241 - acc: 0.3394 - val_loss: 1.2176 - val_acc: 0.4944
Epoch 4/10
50/50 [=====

In [18]:
with open('./trainHistoryDict_2nd', 'wb') as file_pi:
    pickle.dump(vgghist.history, file_pi)